<a href="https://colab.research.google.com/github/iammtg/youtube-live-chat/blob/master/YouTube_chat_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install yt-dlp

In [ ]:
youtube_url = "https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=11675s"#@param {type:"string"}

In [ ]:
import re
def remove_timestamp_from_youtube_url(url):
    # 正規表現パターンでタイムスタンプを検索して除去する
    pattern = r'(&|\?)t=\d+s?'
    clean_url = re.sub(pattern, '', url)
    return clean_url
youtube_url = remove_timestamp_from_youtube_url(youtube_url)

In [ ]:
!yt-dlp --cookies /content/cookies.txt --write-sub --sub-lang live_chat --skip-download $youtube_url -o livechat

[youtube] Extracting URL: https://www.youtube.com/watch?v=Vk0w-bgUrUA
[youtube] Vk0w-bgUrUA: Downloading webpage
[youtube] Vk0w-bgUrUA: Downloading ios player API JSON
[youtube] Vk0w-bgUrUA: Downloading m3u8 information
[info] Vk0w-bgUrUA: Downloading subtitles: live_chat
[info] Vk0w-bgUrUA: Downloading 1 format(s): 303+251
Deleting existing file livechat.live_chat.json
[info] Writing video subtitles to: livechat.live_chat.json
[youtube_live_chat] Downloading live chat
[youtube_live_chat] Total fragments: unknown (live)
[download] Destination: livechat.live_chat.json
[download] 100% of   15.10MiB in 00:00:13 at 1.10MiB/s


In [ ]:
import json
filename = '/content/livechat.live_chat.json'
output_filename = "chat.txt"  # 出力するファイル名を設定

def save_messages_to_file(filename_in, filename_out):
    try:
        with open(filename_in, 'r', encoding='utf-8') as f_in, open(filename_out, 'w', encoding='utf-8') as f_out:
            lines = f_in.readlines()

            for line in lines:
                try:
                    data = json.loads(line.strip())
                    if 'liveChatTextMessageRenderer' in data.get('replayChatItemAction', {}).get('actions', [{}])[0].get('addChatItemAction', {}).get('item', {}):
                        message = data['replayChatItemAction']['actions'][0]['addChatItemAction']['item']['liveChatTextMessageRenderer']['message']['runs'][0]['text']
                        timestamp = data['replayChatItemAction']['actions'][0]['addChatItemAction']['item']['liveChatTextMessageRenderer']['timestampText']['simpleText']
                        f_out.write(f"{timestamp}\t{message}\n")
                        #print(f"{timestamp}\t{message}")
                    else:
                        # f_out.write("No message found.\n")
                        pass
                except json.JSONDecodeError as e:
                    print(f"JSONDecodeError on line: {line.strip()}")
                    print(f"Error: {e.msg} at line {e.lineno}, column {e.colno}")
                except Exception as e:
                    # print(f"Unexpected error: {e}")
                    pass

    except Exception as e:
        print(f"Error: {e}")

# メッセージとタイムスタンプをファイルに保存
save_messages_to_file(filename, output_filename)

print(f"Messages saved to {output_filename}.")

Messages saved to chat.txt.


In [ ]:
search_words = "助かる ASMR" # @param {type:"string"}
search_strings = search_words.split(' ')
search_mode = "AND" # @param ["OR", "AND"]

In [ ]:
def convert_timestamp_to_seconds(timestamp):
    parts = timestamp.split(':')
    if len(parts) == 2:
        hours, minutes = 0, int(parts[0])
        seconds = int(parts[1])
    elif len(parts) == 3:
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
    else:
        raise ValueError("Invalid timestamp format")

    total_seconds = hours * 3600 + minutes * 60 + seconds - 15
    return total_seconds

def search_strings_in_file(file_path, search_strings, search_mode='OR', youtube_url=''):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        if search_mode.upper() == 'OR':
            matching_lines = [line.strip() for line in lines if any(search_string in line for search_string in search_strings)]
        elif search_mode.upper() == 'AND':
            matching_lines = [line.strip() for line in lines if all(search_string in line for search_string in search_strings)]
        else:
            matching_lines = [line.strip() for line in lines]

        if matching_lines:
            print(f"Lines containing {search_mode} of {search_strings}:")
            for line in matching_lines:
                parts = line.split('\t')
                timestamp = parts[0]
                content = '\t'.join(parts[1:])

                # Convert timestamp to seconds
                seconds = convert_timestamp_to_seconds(timestamp)

                # Generate YouTube URL
                youtube_link = f"{youtube_url}&t={seconds}"

                # Print line with YouTube URL
                print(f"{timestamp} {content} {youtube_link}")
        else:
            print(f"No lines containing {search_mode} of {search_strings} found.")

    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# 検索文字列と検索モードを指定して関数を呼び出す
search_mode = 'OR'  # 'OR' または 'AND' を指定
search_strings_in_file(output_filename, search_strings, search_mode, youtube_url)

Lines containing OR of ['助かる', 'ASMR']:
0:21 神OP助かる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=6
33:36 はいASMR開始ー https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=2001
41:20 コンビーフ助かる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=2465
59:09 ASMRうおおおおおおお https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3534
59:15 ASMRきちゃ https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3540
59:16 ASMR！ https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3541
59:21 ASMRたすかる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3546
59:23 ASMRきちゃー https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3548
59:23 ASMRたすかる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=3548
1:07:58 FA助かる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=4063
1:30:57 バランスボールASMR? https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=5442
1:32:55 しゅっしゅっ助かる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=5560
1:48:36 しゅっしゅん助かる https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=6501
1:49:54 左下にASMRが https://www.youtube.com/watch?v=Vk0w-bgUrUA&t=6579
1:50:16 夜見のASMRもききたいね https: